In [3]:
from gurobipy import *

In [4]:
poop = gurobipy.Model("poop")

In [5]:
#THE FOLLOWING CODE COMES FROM: http://www.gurobi.com/documentation/8.0/examples/multiobj_py.html#subsubsection:multiobj.py

#!/usr/bin/python

# Copyright 2018, Gurobi Optimization, LLC

# Want to cover three different sets but subject to a common budget of
# elements allowed to be used. However, the sets have different priorities to
# be covered; and we tackle this by using multi-objective optimization.

from __future__ import print_function
from gurobipy import *

try:
    # Sample data
    Groundset = range(20)
    Subsets   = range(4)
    Budget    = 12;
    Set = [ [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
            [ 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1 ],
            [ 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0 ],
            [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0 ] ]
    SetObjPriority = [  3,    2,    2,   1]
    SetObjWeight   = [1.0, 0.25, 1.25, 1.0]

    # Create initial model
    model = Model('multiobj')

    # Initialize decision variables for ground set:
    # x[e] == 1 if element e is chosen for the covering.
    Elem = model.addVars(Groundset, vtype=GRB.BINARY, name='El')

    # Constraint: limit total number of elements to be picked to be at most
    # Budget
    model.addConstr(Elem.sum() <= Budget, name='Budget')

    # Set global sense for ALL objectives
    model.ModelSense = GRB.MAXIMIZE

    # Limit how many solutions to collect
    model.setParam(GRB.Param.PoolSolutions, 100)

    # Set and configure i-th objective
    for i in Subsets:
        objn = sum(Elem[k]*Set[i][k] for k in range(len(Elem)))
        model.setObjectiveN(objn, i, SetObjPriority[i], SetObjWeight[i],
                            1.0 + i, 0.01, 'Set' + str(i))

    # Save problem
    model.write('multiobj.lp')

    # Optimize
    model.optimize()

    model.setParam(GRB.Param.OutputFlag, 0)

    # Status checking
    status = model.Status
    if status == GRB.Status.INF_OR_UNBD or \
       status == GRB.Status.INFEASIBLE  or \
       status == GRB.Status.UNBOUNDED:
        print('The model cannot be solved because it is infeasible or unbounded')
        sys.exit(1)

    if status != GRB.Status.OPTIMAL:
        print('Optimization was stopped with status ' + str(status))
        sys.exit(1)

    # Print best selected set
    print('Selected elements in best solution:')
    for e in Groundset:
        if Elem[e].X > 0.9:
            print(' El%d' % e, end='')
    print('')

    # Print number of solutions stored
    nSolutions = model.SolCount
    print('Number of solutions found: ' + str(nSolutions))

    # Print objective values of solutions
    if nSolutions > 10:
        nSolutions = 10
    print('Objective values for first ' + str(nSolutions) + ' solutions:')
    for i in Subsets:
        model.setParam(GRB.Param.ObjNumber, i)
        print('\tSet%d' % i, end='')
        for e in range(nSolutions):
            model.setParam(GRB.Param.SolutionNumber, e)
            print(' %6g' % model.ObjNVal, end='')
        print('')

except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError as e:
    print('Encountered an attribute error: ' + str(e))

Changed value of parameter PoolSolutions to 100
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (3 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve time: 0.02s
Presolved: 1 rows and 20 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (Set0) ...
---------------------------------------------------------------------------

Found heuristic solu

# Now let's try to implement our model in gurobipy

In [27]:
def distance(a,b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    return math.sqrt(dx*dx + dy*dy)

In [36]:
#Problem Data

#FILL IN LATER
demandSites = [[1,2],[3,4]]
demandWt = [20,30]
potlFacilitySites = [[5,6],[10,4],[0,1]]

# potl covering distance for each site is the same for
# us in this simple initial try
S = 5.0 
#number of facilities to place
P=1

In [37]:
numDemandSites = len(demandSites)
numPotlFacilitySites = len(potlFacilitySites)

In [38]:
m = Model('C-MCLP-v0.0')

In [39]:
# X : POTL FACILITY SITES
x = m.addVars(range(numPotlFacilitySites), vtype=GRB.BINARY, name="x")
e = m.addVars(range(numPotlFacilitySites), vtype=GRB.BINARY, name="e")
y = m.addVars(range(numDemandSites), vtype=GRB.CONTINUOUS, name="y")

#Add Variables
#x = {}
#y = {}
#e = {}

#for j in range(numPotlFacilitySites):
#    x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)
#    e[j] = m.addVar(vtype=GRB.BINARY, name="e%d" % j)
    
#for i in range(numDemandSites):
#    y[i] = m.addVar(vtype=GRB.CONTINUOUS, name="y%d" % i)

In [40]:
# N[i] is the covering set of i (all j that can cover i)
N = {}
for i in range(numDemandSites):
    N[i] = []
    for j in range(numPotlFacilitySites):
        if distance(demandSites[i], potlFacilitySites[j]) <= S:
            N[i].append(j)
            

In [41]:
# Add Constraints

#TODO - not sure what this does/where it should go
m.update()

for i in range(numDemandSites):
    #c1
    m.addConstr(quicksum(x[j] + y[i] for j in N[i]) >= 1)
    #c3
    m.addConstr(quicksum(x[j] + e[j] for j in N[i]) >= 1)
    
#c2
#m.addConstr(quicksum(x[j] for j in range(numPotlFacilitySites) == P))
m.addConstr(x.sum() == P)

#c4
# x[j] is 0 or 1
#c5
# e[j] is 0 or 1

#c6
for i in range(numDemandSites):
    m.addConstr(y[i] >= 0)


In [44]:
# Set global sense for ALL objectives
m.ModelSense = GRB.MINIMIZE

# Set Objectives
    #expr: New alternative objective.
    #index: Index for new objective. If you use an index of 0, this routine will change the primary optimization objective.
    #priority: Priority for the alternative objective.
    #weight: Weight for the alternative objective.
    #abstol: Absolute tolerance for the alternative objective. 
    #reltol: Relative tolerance for the alternative objective. 
    #name: Name of the alternative objective. 

#o1- min amt of demand that is NOT covered
m.setObjectiveN(quicksum(y[i]*demandWt[i] for i in range(numDemandSites)),
                    1, 1, 0.5, 2.0, 0.01,"Objective 1")

#o2 - (while) min'g w/e you need to add to cover everything
m.setObjectiveN(quicksum(e[j] for j in range(numPotlFacilitySites)),
                    2, 2, 0.5, 3.0, 0.01, "Objective 2")


In [48]:
try:
    m.optimize()
    m.setParam(GRB.Param.OutputFlag, 0)

    # Status checking
    status = m.Status
    if status == GRB.Status.INF_OR_UNBD or \
        status == GRB.Status.INFEASIBLE  or \
        status == GRB.Status.UNBOUNDED:
        print('The model cannot be solved because it is infeasible or unbounded')
        sys.exit(1)

    if status != GRB.Status.OPTIMAL:
        print('Optimization was stopped with status ' + str(status))
        sys.exit(1)

    # Print best selected set
    print('Selected elements in best solution:')
    for j in range(numPotlFacilitySites):
        if x[j].X > 0.9:
            print(' x%d' % j, end='')
        print('')

    # Print number of solutions stored
    nSolutions = m.SolCount
    print('Number of solutions found: ' + str(nSolutions))

    # Print objective values of solutions
    if nSolutions > 10:
        nSolutions = 10
    print('Objective values for first ' + str(nSolutions) + ' solutions:')
    for i in range(2): #num of objectives
        m.setParam(GRB.Param.ObjNumber, i)
        print('\tObj%d' % i, end='')
        for e in range(nSolutions):
            m.setParam(GRB.Param.SolutionNumber, e)
            print(' %6g' % m.ObjNVal, end='')
        print('')

except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError as e:
    print('Encountered an attribute error: ' + str(e))

Selected elements in best solution:


 x2
Number of solutions found: 1
Objective values for first 1 solutions:
	Obj0      0
	Obj1      0


In [49]:
#N {demand_site_i: [facility_site_j that cover demand_site_i,...],...}
N

{0: [2], 1: [0, 2]}

In [ ]:
# so thhis checks out because3 we successfully picked out x[2] 
# (remember: x[j]==1 when we DO place site, 0 if not; e[j] is opp.)